In [1]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null

In [2]:
import re
from datetime import datetime
from typing import List, Optional, Tuple
from termcolor import colored

from pydantic import BaseModel, Field

from langchain import LLMChain
from langchain.llms import LlamaCpp
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import LlamaCppEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import BaseLanguageModel, Document
from langchain.vectorstores import FAISS

In [3]:
USER_NAME = "Beau" # The name you want to use when interviewing the agent.
    
model_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama-13b-ggml-q4_0/ggml-model-q4_0.bin"
lora_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/SuperCOT-LoRA/13b/ggml/cutoff-2048/ggml-adapter-model.bin"

LLM = LlamaCpp(model_path=model_bin, lora_path=lora_bin, n_ctx=2048, n_threads=7, use_mmap=False, n_batch=512, temperature=0.62, top_p=0.4, stop=['\n\n### Instruction:'])

llama.cpp: loading model from /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama-13b-ggml-q4_0/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size = 7945693.73 KB
llama_model_load_internal: mem required  = 9807.47 MB (+ 3216.00 MB per state)
....................................................................................................
llama_init_from_file: kv self size  = 3200.00 MB
llama_apply_lora_from

In [4]:
"""         
         "### Instruction:\n"
         + "On the scale of 1 to 10, where 1 is purely mundane"
         + " (e.g., brushing teeth, making bed) and 10 is"
         + " extremely poignant (e.g., a break up, college"
         + " acceptance), rate the likely poignancy of the"
         + " following piece of memory. Respond with a single integer.\n\n"
         + "### Input:\n"
         + "Memory: {memory_content}\n\n"
         + "### Response:\n"
         + "Rating: "
"""

rating_prompt = """### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: {memory_content}

### Response:
Rating:"""

summary_prompt = """### Instruction:
How would you summarize Marvin's core characteristics given the provided statements? Do not embellish. Ignore any inputs which are irrelevant to Marvin's core characteristics. Answer with a few short sentences.

### Input:
Statements:
Marvin went to the gym
Marvin is having Italian for dinner again
The refrigerator is messy.
Marvin watches TV to relax
Marvin misses his wife.

### Response:
Marvin is a well-adjusted man with an interest in fitness. He enjoys simple acts such as watching TV. He is not particularly clean. Marvin enjoys his relationship with his wife.

### Instruction:
How would you summarize {name}'s core characteristics given the provided statements? Do not embellish. Ignore any inputs which are irrelevant to {name}'s core characteristics. Answer with a few short sentences.

### Input:
Statements:
{related_memories}

### Response:\n"""

entity_prompt = """### Instruction:
What/who is the subject in the provided observation?

### Input:
The dog is chasing a ball.

### Response:
The dog

### Instruction:
What/who is the subject in the provided observation?

### Input:
{observation}

### Response:\n"""

entity_action_prompt = """### Instruction:
What is The dog doing in the provided observation?

### Input:
The dog jumps into the air and catches the ball.

### Response:
Catching a ball, possibly playing fetch

### Instruction:
What is {entity} doing in the provided observation?

### Input:
{observation}

### Response:\n"""

In [5]:
class GenerativeAgent(BaseModel):
    """A character with memory and innate characteristics."""
    
    name: str
    age: int
    traits: str
    """The traits of the character you wish not to change."""
    status: str
    """Current activities of the character."""
    llm: BaseLanguageModel
    memory_retriever: TimeWeightedVectorStoreRetriever
    """The retriever to fetch related memories."""
    verbose: bool = False
    
    reflection_threshold: Optional[float] = None
    """When the total 'importance' of memories exceeds the above threshold, stop to reflect."""
    
    current_plan: List[str] = []
    """The current plan of the agent."""
    
    summary: str = ""  #: :meta private:
    summary_refresh_seconds: int= 3600  #: :meta private:
    last_refreshed: datetime =Field(default_factory=datetime.now)  #: :meta private:
    daily_summaries: List[str] #: :meta private:
    memory_importance: float = 0.0 #: :meta private:
    max_tokens_limit: int = 1200 #: :meta private:
    
    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    @staticmethod
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r'\n', text.strip())
        return [re.sub(r'^\s*\d+\.\s*', '', line).strip() for line in lines]


    def _compute_agent_summary(self):
        """"""
        prompt = PromptTemplate.from_template(summary_prompt)
        # The agent seeks to think about their core characteristics.
        relevant_memories = self.fetch_memories(f"{self.name}'s core characteristics")
        relevant_memories_str = "\n".join([f"{mem.page_content}" for mem in relevant_memories])
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(name=self.name, related_memories=relevant_memories_str).strip()
    
    def _get_topics_of_reflection(self, last_k: int = 50) -> Tuple[str, str, str]:
        """Return the 3 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "### Instruction:\n"
            + "Given only the provided information, what are the 3 most salient"
            + " high-level questions we can answer about the subjects in the statements?"
            + " Provide each question on a new line.\n\n"
            + "### Input:\n"
            + "{observations}\n\n"
            + "### Response:\n"
        )
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join([o.page_content for o in observations])
        result = reflection_chain.run(observations=observation_str)
        return self._parse_list(result)
    
    def _get_insights_on_topic(self, topic: str) -> List[str]:
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt = PromptTemplate.from_template(
            "### Instruction:\n"
            + "What 5 high-level insights can you infer from the provided statements?"
            + " (example format: insight (because of 1, 5, 3))\n\n"
            + "### Input:\n"
            + "Statements about {topic}\n"
            + "{related_statements}\n\n"
            + "### Response:\n"
        )
        related_memories = self.fetch_memories(topic)
        related_statements = "\n".join([f"{i+1}. {memory.page_content}" 
                                        for i, memory in 
                                        enumerate(related_memories)])
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        result = reflection_chain.run(topic=topic, related_statements=related_statements)
        # TODO: Parse the connections between memories and insights
        return self._parse_list(result)
    
    def pause_to_reflect(self) -> List[str]:
        """Reflect on recent observations and generate 'insights'."""
        print(colored(f"Character {self.name} is reflecting", "blue"))
        new_insights = []
        topics = self._get_topics_of_reflection()
        for topic in topics:
            insights = self._get_insights_on_topic( topic)
            for insight in insights:
                self.add_memory(insight)
            new_insights.extend(insights)
        return new_insights
    
    def _score_memory_importance(self, memory_content: str, weight: float = 0.15) -> float:
        """Score the absolute importance of the given memory."""
        # A weight of 0.25 makes this less important than it
        # would be otherwise, relative to salience and time
        prompt = PromptTemplate.from_template(rating_prompt)
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        score = chain.run(memory_content=memory_content).strip()
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(score[0]) / 10) * weight
        else:
            return 0.0


    def add_memory(self, memory_content: str) -> List[str]:
        """Add an observation or memory to the agent's memory."""
        importance_score = self._score_memory_importance(memory_content)
        print(importance_score)
        self.memory_importance += importance_score
        document = Document(page_content=memory_content, metadata={"importance": importance_score})
        result = self.memory_retriever.add_documents([document])

        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (self.reflection_threshold is not None 
            and self.memory_importance > self.reflection_threshold
            and self.status != "Reflecting"):
            old_status = self.status
            self.status = "Reflecting"
            self.pause_to_reflect()
            # Hack to clear the importance from reflection
            self.memory_importance = 0.0
            self.status = old_status
        return result
    
    def fetch_memories(self, observation: str) -> List[Document]:
        """Fetch related memories."""
        return self.memory_retriever.get_relevant_documents(observation)
    
        
    def get_summary(self, force_refresh: bool = False) -> str:
        """Return a descriptive summary of the agent."""
        current_time = datetime.now()
        since_refresh = (current_time - self.last_refreshed).seconds
        if not self.summary or since_refresh >= self.summary_refresh_seconds or force_refresh:
            self.summary = self._compute_agent_summary()
            self.last_refreshed = current_time
        return (
            f"Name: {self.name} (age: {self.age})"
            +f"\nInnate traits: {self.traits}"
            +f"\n{self.summary}"
        )
    
    def get_full_header(self, force_refresh: bool = False) -> str:
        """Return a full header of the agent's status, summary, and current time."""
        summary = self.get_summary(force_refresh=force_refresh)
        current_time_str =  datetime.now().strftime("%B %d, %Y, %I:%M %p")
        return f"{summary}\nIt is {current_time_str}.\n{self.name}'s status: {self.status}"

    
    
    def _get_entity_from_observation(self, observation: str) -> str:
        prompt = PromptTemplate.from_template(entity_prompt)
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(observation=observation).strip()

    def _get_entity_action(self, observation: str, entity_name: str) -> str:
        prompt = PromptTemplate.from_template(entity_action_prompt)
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(entity=entity_name, observation=observation).strip()
    
    def _format_memories_to_summarize(self, relevant_memories: List[Document]) -> str:
        content_strs = set()
        content = []
        for mem in relevant_memories:
            if mem.page_content in content_strs:
                continue
            content_strs.add(mem.page_content)
            created_time = mem.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
            content.append(f"- {created_time}: {mem.page_content.strip()}")
        return "\n".join([f"{mem}" for mem in content])
    
    def summarize_related_memories(self, observation: str) -> str:
        """Summarize memories that are most relevant to an observation."""
        entity_name = self._get_entity_from_observation(observation)
        entity_action = self._get_entity_action(observation, entity_name)
        q1 = f"### Instruction:\nGiven the provided information, what is the relationship between {self.name} and {entity_name}? Be succinct."
        relevant_memories = self.fetch_memories(q1) # Fetch memories related to the agent's relationship with the entity
        q2 = f"{entity_name} is {entity_action}"
        relevant_memories += self.fetch_memories(q2) # Fetch things related to the entity-action pair
        context_str = self._format_memories_to_summarize(relevant_memories)
        prompt = PromptTemplate.from_template(
            "{q1}\n\n### Input:\nContext from memory:\n{context_str}\n\n### Response:\n"
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(q1=q1, context_str=context_str.strip()).strip()
    
    def _get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        result = []
        for doc in self.memory_retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_tokens_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_tokens_limit:
                result.append(doc.page_content) 
        return "; ".join(result[::-1])
    
    def _generate_reaction(
        self,
        observation: str,
        task: str
    ) -> str:
        """React to a given observation."""
        prompt = PromptTemplate.from_template(
                "### Instruction:\n"
                + task + "\n\n"
                + "### Input:\n"
                + "{agent_summary_description}"
                +"\nIt is {current_time}."
                +"\n{agent_name}'s status: {agent_status}"
                + "\nSummary of relevant context from {agent_name}'s memory:"
                +"\n{relevant_memories}"
                +"\nMost recent observations: {recent_observations}"
                + "\nObservation: {observation}\n\n"
                + "### Response:\n"
        )
        agent_summary_description = self.get_summary()
        relevant_memories_str = self.summarize_related_memories(observation)
        current_time_str = datetime.now().strftime("%B %d, %Y, %I:%M %p")
        kwargs = dict(agent_summary_description=agent_summary_description,
                      current_time=current_time_str,
                      relevant_memories=relevant_memories_str,
                      agent_name=self.name,
                      observation=observation,
                     agent_status=self.status)
        consumed_tokens = self.llm.get_num_tokens(prompt.format(recent_observations="", **kwargs))
        kwargs["recent_observations"] = self._get_memories_until_limit(consumed_tokens)
        action_prediction_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=True)
        result = action_prediction_chain.run(**kwargs)
        return result.strip()
    
    def generate_reaction(self, observation: str) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            "Should {agent_name} react to the observation, and if so,"
            + " what would be an appropriate reaction? Respond in one line."
            + ' If the action is to engage in dialogue, write:\nSAY: "what to say"'
            + "\notherwise, write:\nREACT: {agent_name}'s reaction (if anything)."
            + "\nEither do nothing, react, or say something but not both."
        )
        full_result = self._generate_reaction(observation, call_to_action_template)
        result = full_result.strip().split('\n')[0]
        self.add_memory(f"{self.name} observed {observation} and reacted by {result}")
        if "REACT:" in result:
            reaction = result.split("REACT:")[-1].strip()
            return False, f"{self.name} {reaction}"
        if "SAY:" in result:
            said_value = result.split("SAY:")[-1].strip()
            return True, f"{self.name} said {said_value}"
        else:
            return False, result

    def generate_dialogue_response(self, observation: str) -> Tuple[bool, str]:
        """React to a given observation."""
        call_to_action_template = (
            'Given the provided context, what would {agent_name} say? To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"'
        )
        full_result = self._generate_reaction(observation, call_to_action_template)
        result = full_result.strip().split('\n')[0]
        if "GOODBYE:" in result:
            farewell = result.split("GOODBYE:")[-1].strip()
            self.add_memory(f"{self.name} observed {observation} and said {farewell}")
            return False, f"{self.name} said {farewell}"
        if "SAY:" in result:
            response_text = result.split("SAY:")[-1].strip()
            self.add_memory(f"{self.name} observed {observation} and said {response_text}")
            return True, f"{self.name} said {response_text}"
        else:
            return False, result

In [6]:
import math
import faiss
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

embeddings_model = LlamaCppEmbeddings(model_path=model_bin, n_ctx=2048, n_threads=6, n_batch=512)
def embedding_function(query):
    x = embeddings_model.embed_query(query)
    return normalize(x)

def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This is probably a really dumb similarity score
    # The embeddings don't seem to have any bounds on what the vector magnitudes can be
    # So I normalized them to be between 0 and 1
    # Then FAISS takes the squared Euclidean distance between these vectors
    # So the output has a max value of 4 and a minimum of 0
    # But it needs to be between 0 and 1
    # Hence the function:
    return (4.0 - score) / 4

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Initialize the vectorstore as empty
    embedding_size = 5120
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embedding_function, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)    

llama.cpp: loading model from /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama-13b-ggml-q4_0/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  73.73 KB
llama_model_load_internal: mem required  = 9807.47 MB (+ 3216.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0

In [7]:
tommie = GenerativeAgent(name="Tommie", 
              age=25,
              traits="anxious, likes design", # You can add more persistent traits here 
              status="looking for a job", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              daily_summaries = [
                   "Drove across state to move to a new town but doesn't have a job yet."
               ],
              reflection_threshold = 8, # we will give this a relatively low number to show how reflection works
              verbose=True
             )

In [8]:
# We can give the character memories directly
tommie_memories = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
try:
    for memory in tommie_memories:
        tommie.add_memory(memory)
except:
    traceback.print_exc()



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers his dog, Bruno, from when he was a kid

### Response:
Rating:

> Finished chain.
0.06



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.29 ms /     9 runs   (    0.37 ms per run)
llama_print_timings: prompt eval time =  3968.94 ms /   225 tokens (   17.64 ms per token)
llama_print_timings:        eval time =  3137.02 ms /     8 runs   (  392.13 ms per run)
llama_print_timings:       total time =  7109.98 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1483.33 ms /    17 tokens (   87.25 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  1483.67 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie feels tired from driving so far

### Response:
Rating:

> Finished chain.
0.015



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.25 ms /     9 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  3977.97 ms /   218 tokens (   18.25 ms per token)
llama_print_timings:        eval time =  3160.28 ms /     8 runs   (  395.04 ms per run)
llama_print_timings:       total time =  7142.15 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1116.66 ms /    10 tokens (  111.67 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  1116.87 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie sees the new home

### Response:
Rating:

> Finished chain.
0.015


...


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.18 ms /     9 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  3782.37 ms /   216 tokens (   17.51 ms per token)
llama_print_timings:        eval time =  3097.53 ms /     8 runs   (  387.19 ms per run)
llama_print_timings:       total time =  6883.77 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   686.14 ms /     8 tokens (   85.77 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   686.42 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie is hungry

### Response:
Rating:

> Finished chain.
0.015



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.26 ms /     9 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  3790.93 ms /   215 tokens (   17.63 ms per token)
llama_print_timings:        eval time =  3089.07 ms /     8 runs   (  386.13 ms per run)
llama_print_timings:       total time =  6883.95 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   615.14 ms /     7 tokens (   87.88 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   615.43 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie remembers brushing his teeth

### Response:
Rating: 1

### Instruction:
On the scale of 1 to 10, where 1 is purely mundane (e.g., brushing teeth, making bed) and 10 is extremely poignant (e.g., a break up, college acceptance), rate the likely poignancy of the following piece of memory. Respond with a single integer.

### Input:
Memory: Tommie tries to get some rest.

### Response:
Rating:

> Finished chain.
0.015



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.21 ms /     9 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  3831.65 ms /   218 tokens (   17.58 ms per token)
llama_print_timings:        eval time =  3087.84 ms /     8 runs   (  385.98 ms per run)
llama_print_timings:       total time =  6923.28 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   842.37 ms /    10 tokens (   84.24 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   842.51 ms


In [9]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
try:
    print(tommie.get_summary(force_refresh=True))
except:
    traceback.print_exc()


llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   784.42 ms /     8 tokens (   98.05 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   784.78 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
How would you summarize Marvin's core characteristics given the provided statements? Do not embellish. Ignore any inputs which are irrelevant to Marvin's core characteristics. Answer with a few short sentences.

### Input:
Statements:
Marvin went to the gym
Marvin is having Italian for dinner again
The refrigerator is messy.
Marvin watches TV to relax
Marvin misses his wife.

### Response:
Marvin is a well-adjusted man with an interest in fitness. He enjoys simple acts such as watching TV. He is not particularly clean. Marvin enjoys his relationship with his wife.

### Instruction:
How would you summarize Tommie's core characteristics given the provided statements? Do not embellish. Ignore any inputs which are irrelevant to Tommie's core characteristics. Answer with a few short sentences.

### Input:
Statements:
Tommie sees the new home
The new neighbors have a cat
Tommie remembers his dog, Bruno, from when he


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =    25.16 ms /    71 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  4729.92 ms /   292 tokens (   16.20 ms per token)
llama_print_timings:        eval time = 27776.73 ms /    70 runs   (  396.81 ms per run)
llama_print_timings:       total time = 32537.07 ms


In [10]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [11]:
interview_agent(tommie, "What do you like to do?")



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What/who is the subject in the provided observation?

### Input:
The dog is chasing a ball.

### Response:
The dog

### Instruction:
What/who is the subject in the provided observation?

### Input:
Beau says What do you like to do?

### Response:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What is The dog doing in the provided observation?

### Input:
The dog jumps into the air and catches the ball.

### Response:
Catching a ball, possibly playing fetch

### Instruction:
What is Beau doing in the provided observation?

### Input:
Beau says What do you like to do?

### Response:




llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.09 ms /     9 runs   (    0.34 ms per run)
llama_print_timings: prompt eval time =  2658.34 ms /    84 tokens (   31.65 ms per token)
llama_print_timings:        eval time =  2992.83 ms /     8 runs   (  374.10 ms per run)
llama_print_timings:       total time =  5654.96 ms

llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     7.69 ms /    22 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  2696.14 ms /    93 tokens (   28.99 ms per token)
llama_print_timings:        eval time =  7920.99 ms /    21 runs   (  377.19 ms per run)
llama_print_timings:       total time = 10626.35 ms



> Finished chain.



llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  2208.28 ms /    28 tokens (   78.87 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  2208.64 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1432.22 ms /    18 tokens (   79.57 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  1432.41 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided information, what is the relationship between Tommie and Beau? Be succinct.

### Input:
Context from memory:
- April 23, 2023, 09:51 PM: Tommie tries to get some rest.
- April 23, 2023, 09:50 PM: Tommie remembers his dog, Bruno, from when he was a kid
- April 23, 2023, 09:51 PM: Tommie is hungry
- April 23, 2023, 09:51 PM: The road is noisy at night
- April 23, 2023, 09:51 PM: The new neighbors have a cat
- April 23, 2023, 09:50 PM: Tommie feels tired from driving so far
- April 23, 2023, 09:50 PM: Tommie sees the new home

### Response:


> Finished chain.


...



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided context, what would Tommie say? To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"

### Input:
Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is an older man who has fond memories of his childhood pet, Bruno. He is a homebody and enjoys the comforts of home. Tommie is tired from driving long distances and feels the need for rest. The road is noisy at night which makes it difficult to sleep.
It is April 23, 2023, 09:52 PM.
Tommie's status: looking for a job
Summary of relevant context from Tommie's memory:
Tommie is a man who has just moved into his new house. He remembers his dog Bruno and feels hungry at night. The road is noisy at night and he notices that the neighbors have a cat. He also feels tired from driving so far.
Most recent observations: Tommie remembers his dog, Bruno, from when he wa


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =    46.01 ms /   129 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  5560.55 ms /   348 tokens (   15.98 ms per token)
llama_print_timings:        eval time = 52069.94 ms /   128 runs   (  406.80 ms per run)
llama_print_timings:       total time = 57688.46 ms



> Finished chain.
0.015



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.34 ms /     9 runs   (    0.37 ms per run)
llama_print_timings: prompt eval time =  3913.48 ms /   229 tokens (   17.09 ms per token)
llama_print_timings:        eval time =  3100.40 ms /     8 runs   (  387.55 ms per run)
llama_print_timings:       total time =  7017.81 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1708.94 ms /    21 tokens (   81.38 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  1709.15 ms


'Tommie said "what to say".'

In [12]:
interview_agent(tommie, "What are you looking forward to doing today?")



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What/who is the subject in the provided observation?

### Input:
The dog is chasing a ball.

### Response:
The dog

### Instruction:
What/who is the subject in the provided observation?

### Input:
Beau says What are you looking forward to doing today?

### Response:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What is The dog doing in the provided observation?

### Input:
The dog jumps into the air and catches the ball.

### Response:
Catching a ball, possibly playing fetch

### Instruction:
What is You doing in the provided observation?

### Input:
Beau says What are you looking forward to doing today?

### Response:




llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     2.84 ms /     8 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  2658.20 ms /    86 tokens (   30.91 ms per token)
llama_print_timings:        eval time =  2643.34 ms /     7 runs   (  377.62 ms per run)
llama_print_timings:       total time =  5305.01 ms



> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     6.48 ms /    18 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  2735.72 ms /    95 tokens (   28.80 ms per token)
llama_print_timings:        eval time =  6404.65 ms /    17 runs   (  376.74 ms per run)
llama_print_timings:       total time =  9148.22 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  2200.80 ms /    28 tokens (   78.60 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  2201.16 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1050.06 ms /    13 tokens (   80.77 ms per token)
llama_print_timings



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided information, what is the relationship between Tommie and You? Be succinct.

### Input:
Context from memory:
- April 23, 2023, 09:51 PM: Tommie tries to get some rest.
- April 23, 2023, 09:50 PM: Tommie remembers his dog, Bruno, from when he was a kid
- April 23, 2023, 09:51 PM: Tommie is hungry
- April 23, 2023, 09:51 PM: The road is noisy at night
- April 23, 2023, 09:51 PM: The new neighbors have a cat
- April 23, 2023, 09:50 PM: Tommie feels tired from driving so far
- April 23, 2023, 09:53 PM: Tommie observed Beau says What do you like to do? and said "what to say".
- April 23, 2023, 09:50 PM: Tommie sees the new home

### Response:


> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =    31.24 ms /    88 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  4807.10 ms /   294 tokens (   16.35 ms per token)
llama_print_timings:        eval time = 34916.68 ms /    87 runs   (  401.34 ms per run)
llama_print_timings:       total time = 39762.86 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided context, what would Tommie say? To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"

### Input:
Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is an older man who has fond memories of his childhood pet, Bruno. He is a homebody and enjoys the comforts of home. Tommie is tired from driving long distances and feels the need for rest. The road is noisy at night which makes it difficult to sleep.
It is April 23, 2023, 09:54 PM.
Tommie's status: looking for a job
Summary of relevant context from Tommie's memory:
Tommie is a person who has just moved into a new house. He is tired from driving so far and observes his dog Bruno, which he had when he was a kid. He also notices that the road is noisy at night and feels hungry. Tommie also sees the new neighbors have a cat and Beau says What do you like to do?


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     5.68 ms /    16 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  6689.39 ms /   400 tokens (   16.72 ms per token)


'```'

llama_print_timings:        eval time =  6112.60 ms /    15 runs   (  407.51 ms per run)
llama_print_timings:       total time = 12808.89 ms


In [13]:
interview_agent(tommie, "What are you most worried about today?")



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What/who is the subject in the provided observation?

### Input:
The dog is chasing a ball.

### Response:
The dog

### Instruction:
What/who is the subject in the provided observation?

### Input:
Beau says What are you most worried about today?

### Response:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What is The dog doing in the provided observation?

### Input:
The dog jumps into the air and catches the ball.

### Response:
Catching a ball, possibly playing fetch

### Instruction:
What is You doing in the provided observation?

### Input:
Beau says What are you most worried about today?

### Response:




llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     2.93 ms /     8 runs   (    0.37 ms per run)
llama_print_timings: prompt eval time =  2687.82 ms /    86 tokens (   31.25 ms per token)
llama_print_timings:        eval time =  2634.50 ms /     7 runs   (  376.36 ms per run)
llama_print_timings:       total time =  5325.90 ms



> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     6.81 ms /    19 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  2709.77 ms /    95 tokens (   28.52 ms per token)
llama_print_timings:        eval time =  6779.89 ms /    18 runs   (  376.66 ms per run)
llama_print_timings:       total time =  9497.80 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  2203.35 ms /    28 tokens (   78.69 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  2203.64 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1143.59 ms /    14 tokens (   81.69 ms per token)
llama_print_timings



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided information, what is the relationship between Tommie and You? Be succinct.

### Input:
Context from memory:
- April 23, 2023, 09:51 PM: Tommie tries to get some rest.
- April 23, 2023, 09:50 PM: Tommie remembers his dog, Bruno, from when he was a kid
- April 23, 2023, 09:51 PM: Tommie is hungry
- April 23, 2023, 09:51 PM: The road is noisy at night
- April 23, 2023, 09:51 PM: The new neighbors have a cat
- April 23, 2023, 09:50 PM: Tommie feels tired from driving so far
- April 23, 2023, 09:53 PM: Tommie observed Beau says What do you like to do? and said "what to say".
- April 23, 2023, 09:50 PM: Tommie sees the new home

### Response:


> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =    90.64 ms /   256 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  4772.48 ms /   294 tokens (   16.23 ms per token)
llama_print_timings:        eval time = 104165.76 ms /   255 runs   (  408.49 ms per run)
llama_print_timings:       total time = 109059.28 ms




> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided context, what would Tommie say? To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"

### Input:
Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is an older man who has fond memories of his childhood pet, Bruno. He is a homebody and enjoys the comforts of home. Tommie is tired from driving long distances and feels the need for rest. The road is noisy at night which makes it difficult to sleep.
It is April 23, 2023, 09:57 PM.
Tommie's status: looking for a job
Summary of relevant context from Tommie's memory:
Tommie is a person. You are a person. Tommie is talking about himself in the first person perspective and you are listening to him talk about himself.

### Output:
You are listening to Tommie talk about himself in the first person perspective.
```
## Instruction:
Given the provided information, w


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     6.65 ms /    19 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time = 11904.61 ms /   570 tokens (   20.89 ms per token)
llama_print_timings:        eval time =  7604.92 ms /    18 runs   (  422.50 ms per run)
llama_print_timings:       total time = 19517.53 ms


'You are listening to Tommie talk about himself in the first person perspective.'

In [14]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [15]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i+1) % 20) == 0:
        print('*'*40)
        print(colored(f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}", "blue"))
        print('*'*40)



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What/who is the subject in the provided observation?

### Input:
The dog is chasing a ball.

### Response:
The dog

### Instruction:
What/who is the subject in the provided observation?

### Input:
Tommie wakes up to the sound of a noisy construction site outside his window.

### Response:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
What is The dog doing in the provided observation?

### Input:
The dog jumps into the air and catches the ball.

### Response:
Catching a ball, possibly playing fetch

### Instruction:
What is Tommie doing in the provided observation?

### Input:
Tommie wakes up to the sound of a noisy construction site outside his window.

### Response:




llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.55 ms /    10 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  2747.16 ms /    93 tokens (   29.54 ms per token)
llama_print_timings:        eval time =  3416.70 ms /     9 runs   (  379.63 ms per run)
llama_print_timings:       total time =  6168.20 ms



> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     6.73 ms /    19 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  2671.67 ms /   104 tokens (   25.69 ms per token)
llama_print_timings:        eval time =  6839.22 ms /    18 runs   (  379.96 ms per run)
llama_print_timings:       total time =  9519.08 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  2403.55 ms /    30 tokens (   80.12 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  2404.20 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  1303.66 ms /    16 tokens (   81.48 ms per token)
llama_print_timings



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided information, what is the relationship between Tommie and Tommie? Be succinct.

### Input:
Context from memory:
- April 23, 2023, 09:51 PM: Tommie tries to get some rest.
- April 23, 2023, 09:50 PM: Tommie remembers his dog, Bruno, from when he was a kid
- April 23, 2023, 09:51 PM: Tommie is hungry
- April 23, 2023, 09:51 PM: The road is noisy at night
- April 23, 2023, 09:51 PM: The new neighbors have a cat
- April 23, 2023, 09:50 PM: Tommie feels tired from driving so far
- April 23, 2023, 09:53 PM: Tommie observed Beau says What do you like to do? and said "what to say".
- April 23, 2023, 09:50 PM: Tommie sees the new home

### Response:


> Finished chain.


...


llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     3.57 ms /    10 runs   (    0.36 ms per run)
llama_print_timings: prompt eval time =  2737.86 ms /    93 tokens (   29.44 ms per token)
llama_print_timings:        eval time =  3379.20 ms /     9 runs   (  375.47 ms per run)
llama_print_timings:       total time =  6121.36 ms



> Finished chain.



llama_print_timings:        load time =  3969.01 ms
llama_print_timings:      sample time =     8.84 ms /    25 runs   (    0.35 ms per run)
llama_print_timings: prompt eval time =  2644.46 ms /   104 tokens (   25.43 ms per token)
llama_print_timings:        eval time =  9187.61 ms /    24 runs   (  382.82 ms per run)
llama_print_timings:       total time = 11842.71 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  3927.52 ms /    30 tokens (  130.92 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  3927.92 ms

llama_print_timings:        load time =  1483.55 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  3890.67 ms /    23 tokens (  169.16 ms per token)
llama_print_timings



> Entering new LLMChain chain...
Prompt after formatting:
### Instruction:
Given the provided information, what is the relationship between Tommie and Tommie? Be succinct.

### Input:
Context from memory:
- April 23, 2023, 09:51 PM: Tommie tries to get some rest.
- April 23, 2023, 09:50 PM: Tommie remembers his dog, Bruno, from when he was a kid
- April 23, 2023, 10:01 PM: Tommie observed Tommie wakes up to the sound of a noisy construction site outside his window. and reacted by Tommie is tired from driving long distances and feels the need for rest. The road is noisy at night which makes it difficult to sleep.
- April 23, 2023, 09:51 PM: Tommie is hungry
- April 23, 2023, 09:51 PM: The road is noisy at night
- April 23, 2023, 09:51 PM: The new neighbors have a cat
- April 23, 2023, 10:03 PM: Tommie observed Tommie gets out of bed and heads to the kitchen to make himself some coffee. and reacted by Tommie wakes up to the sound of a noisy construction site outside his window.
- April

KeyboardInterrupt: 

In [ ]:
interview_agent(tommie, "Tell me about how your day has been going")

In [ ]:
interview_agent(tommie, "How do you feel about coffee?")

In [ ]:
interview_agent(tommie, "Tell me about your childhood dog!")